In [1]:
import numpy as np
import time
from robot.vrep_robot import VrepRobot
from robot.evolved_robot import EvolvedRobot
import vrep.vrep as vrep
from settings import Settings
from vision.tracker import Tracker, get_marker_object
from utility.path_tracking import transform_pos_angle
import math

In [2]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

starting Tracker, video source:  -1


In [3]:
vision_thread.start()

In [4]:
thymio = {
    'name': 'thymio',
    'body': 'Thymio',
    'left_motor': 'leftMotor',
    'right_motor': 'rightMotor',
    'sensor': 'Proximity_sensor',
    'num_sensors': 7
}

settings = Settings(thymio)

Last markers len: 8
11222333444444445Restore Calibration


In [13]:
vrep.simxFinish(-1)
clientID = vrep.simxStart(
    '127.0.0.1',
    19997,
    True,
    True,
    5000,
    5)

if clientID == -1:
    print('Failed connecting to remote API server')
    print('Program ended')

if (vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking) == -1):
    print('Failed to start the simulation\n')
    print('Program ended\n')


OP_MODE = vrep.simx_opmode_oneshot_wait
robot = EvolvedRobot('thymio-II',clientID, None, OP_MODE, None, settings.robot_type)

In [14]:
obstacle = get_marker_object(9)
print(obstacle.realxy())
robot_m = get_marker_object(7)
print(robot_m.realxy())
print(robot_m.orientation())
robot_current_position = robot_m.realxy()[:2]
angle = robot_m.orientation()
print(angle)
if angle >= 0.0 and angle <= math.pi/2:
    theta = (math.pi/2) - angle
else:
    theta = 2*np.pi - angle
    theta += np.pi/2
print(theta)
# update position and orientation of the robot in vrep
position, orientation = transform_pos_angle(
    robot_current_position, theta)
robot.v_set_pos_angle(position, [ 0, 0, theta ])

[0.59639189 0.59535204 0.        ]
[0.48321989 0.50700029 0.        ]
0.0
0.0
1.5707963267948966
